In [1]:
import numpy as np
from scipy.optimize import linprog

# Составим систему, опираясь на имеющиеся данные
## Целевая функция:
<div>
 F-> max = <var>8000x<sub>1</var> &plus; <var>6000x<sub>2</var> &plus; <var>3000x<sub>3</var> <br>
</div>
     
## Система:
<div>
  <var>9x<sub>1</var> &plus; <var>8x<sub>2</var> &plus; <var>3x<sub>3</var> &le; <var>338</var><br>
  <var>10000x<sub>1</var> &plus; <var>7000x<sub>2</var> &plus; <var>5000x<sub>3</var> &le; <var>300000</var><br>
  <var>x<sub>1</var> &#44; <var>x<sub>2</var> &#44; <var>x<sub>3</var> &ge; <var>0</var>
</div>

In [2]:
obj = [8000, 6000, 3000]

rate_of_system = [
    [9, 8, 3],          # area
    [10000, 7000, 5000] # price
]

b = [338,300000]

## Для составления симплекс-таблицы при помощи метода zip() выгрузим коэффициенты из rate_of_system и b в list и присоединим  к ниму оставшийся obj, для дальнейших вычислений

In [3]:
def to_tabl(rate_of_system, b, obj):
    lis = [i + [j] for i, j in zip(rate_of_system, b)]
    f = obj + [0]
    return lis + [f]

## Далее мы проверяем, где именно можно увеличить неосновные значения, не уменьшая значение целевой функции

In [4]:
def can_be_changed(tabl):
    f = tabl[-1]
    return  any(j > 0 for j in f[:-1])

# В случае, если значение целевой фкнкции может быть увеличино, то ищем ее координаты

In [5]:
def get_position(tabl):
    f = tabl[-1]
    column = next(i for i, x in enumerate(f[:-1]) if x > 0)
    
    restrictions = []
    for eq in tabl[:-1]:
        el = eq[column]
        restrictions.append(100000000 if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

# Изменяем базиса, путем вычитания строк и возвращаем получившуюся матрицу

In [12]:
def step(tabl, position):
    new_tabl = [[] for eq in tabl]
    
    i, j = position
    value = tabl[i][j]
    new_tabl[i] = np.array(tabl[i]) / value
    
    for eq_i, eq in enumerate(tabl):
        if eq_i != i:
            multiplier = np.array(new_tabl[i]) * tabl[eq_i][j]
            new_tabl[eq_i] = np.array(tabl[eq_i]) - multiplier
   
    return new_tabl

In [7]:
def is_basic(column):
    return sum(column) == 1 and len([obj for obj in column if obj == 0]) == len(column) - 1

## Алгоритм состоит из трех основных шагов:

### 1) Преобразование системы  в симплекс-таблицу
### 2) Нахождение координат поворота и совершение шага до тех пор, пока мы не пришли к решению
### 3) Преобразование таблицы в решение

In [14]:
def simplex(rate_of_system, b, obj):
    tabl = to_tabl(rate_of_system, b, obj)

    while can_be_changed(tabl):
        position = get_position(tabl)
        tabl = step(tabl, position)
# Извлекаем вектор-решение из симплекс-таблицы
    columns = np.array(tabl).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

In [15]:
solution = simplex(rate_of_system, b, obj)
print('solution: ', solution)

solution:  [2.000000000000007, 39.99999999999999, 0, 0]


# Проверим полученный ответ при помощи метода linprog(method="revised simplex") библиотеки SciPy:

In [16]:
obj = [-8000, -6000, -3000]

rate_of_system = [[9, 8, 3], 
                 [10000, 7000, 5000]]

b = [338, 300000]  

solution = linprog(c=obj, A_ub=rate_of_system, b_ub=b,
              method="revised simplex")

solution

     con: array([], dtype=float64)
     fun: -256000.00000000003
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([0., 0.])
  status: 0
 success: True
       x: array([ 2., 40.,  0.])